In [12]:
import sys
sys.path.append('local_modules') # For using module in colab after cloning it

In [13]:
import random
import numpy as np

from torch.utils.data import DataLoader

In [14]:
# Required Dependency for NJSDE
!pip install git+https://github.com/000Justin000/torchdiffeq.git@jj585

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/000Justin000/torchdiffeq.git (to revision jj585) to /tmp/pip-req-build-tig9jove
  Running command git clone --filter=blob:none --quiet https://github.com/000Justin000/torchdiffeq.git /tmp/pip-req-build-tig9jove
  Running command git checkout -b jj585 --track origin/jj585
  Switched to a new branch 'jj585'
  Branch 'jj585' set up to track remote branch 'jj585' from 'origin'.
  Resolved https://github.com/000Justin000/torchdiffeq.git to commit 66d853a05b498d8180e37cff16117b3bf3c91e64
  Preparing metadata (setup.py) ... done


In [15]:
!git clone https://github.com/Prishat/Unified-TPP-Models.git module

fatal: destination path 'module' already exists and is not an empty directory.


#ATM Dataset

In [16]:
from module.datasets import ATMDataset 
from module.config import atm_params

In [17]:
atm_params

{'seq_len': 10, 'batch_size': 4}

In [18]:
train_set = ATMDataset(atm_params, subset='train')
test_set = ATMDataset(atm_params, subset='test')
train_loader = DataLoader(train_set, batch_size=1024, shuffle=True, collate_fn=ATMDataset.to_features)
test_loader = DataLoader(test_set, batch_size=1024, shuffle=False, collate_fn=ATMDataset.to_features)

100%|██████████| 370102/370102 [00:01<00:00, 244311.26it/s]


TOTAL SEQs: 360417
<0.001 = 0.7278214584853152
<0.01 = 0.8774121207208667
<0.1 = 0.9104131204236579
<1 = 0.9390532544378698
<10 = 0.9831806760152387
<100 = 0.9999567696090351


100%|██████████| 182095/182095 [00:00<00:00, 296667.12it/s]

TOTAL SEQs: 177914
<0.001 = 0.7150568634234472
<0.01 = 0.8972010345793314
<0.1 = 0.9240045468773167
<1 = 0.9480623603125704
<10 = 0.9851347863571714
<100 = 0.9999450860227453


In [19]:
from module.models import rmtpp
from module.config import rmtpp_params

In [20]:
rmtpp_params['epochs'] = 2
rmtpp_params

{'model': 'rmtpp',
 'seq_len': 10,
 'emb_dim': 10,
 'hid_dim': 32,
 'mlp_dim': 16,
 'dropout': 0.1,
 'alpha': 0.05,
 'batch_size': 1024,
 'event_class': 7,
 'verbose_step': 350,
 'importance_weight': 'store_true',
 'lr': 0.001,
 'epochs': 2}

In [21]:
model = rmtpp(rmtpp_params)

In [22]:
model.train(train_loader, test_loader)

100%|██████████| 352/352 [00:06<00:00, 51.28it/s]


time loss:  4.932382851243019
event loss: 0.9560018909829003
total loss: 1.2026210386412484


100%|██████████| 174/174 [00:01<00:00, 92.49it/s]


pcnt=7, rcnt=5
epoch 0
time_error: 1.2832852602005005, PRECISION: 0.6510145829605352, RECALL: 0.8609684495383683, F1: 0.7414144260495276


100%|██████████| 352/352 [00:08<00:00, 43.23it/s]


time loss:  0.869742620927947
event loss: 0.4091173724617277
total loss: 0.45260450380189077


100%|██████████| 174/174 [00:01<00:00, 96.59it/s]

pcnt=7, rcnt=6
epoch 1
time_error: 1.2242984771728516, PRECISION: 0.6707151339895584, RECALL: 0.8309723992651824, F1: 0.7422925898662642


In [26]:
for b in train_loader:
  data = b
  print(b[0][0])
  print(b[1][0])
  break

tensor([0.0000e+00, 4.3981e-04, 1.3542e-03, 1.1343e-03, 2.5463e-04, 2.6620e-04,
        1.7361e-04, 2.7030e+01, 1.2952e+01, 1.3889e-04])
tensor([1, 1, 1, 1, 1, 1, 1, 0, 2, 5])


In [27]:
model.predict(data)

(array([[-0.92794013],
        [-0.11915638],
        [ 1.142068  ],
        ...,
        [ 0.01337127],
        [ 1.1420635 ],
        [-0.9610746 ]], dtype=float32),
 array([1, 3, 5, ..., 4, 5, 0]))

#StackOverflow Dataset

In [ ]:
from module.datasets import stackOverflow

In [ ]:
data = stackOverflow({}, './so/')

In [ ]:
train = stackOverflow.process_rmtpp(data.ts[:100])
test = stackOverflow.process_rmtpp(data.ts[100:200])
train_loader = DataLoader(train, batch_size=4, shuffle=True, collate_fn=stackOverflow.to_features)
test_loader = DataLoader(test, batch_size=4, shuffle=False, collate_fn=stackOverflow.to_features)

In [ ]:
from module.models import rmtpp
from module.config import rmtpp_params

In [ ]:
rmtpp_params['event_class'] = 22
rmtpp_params['epochs'] = 2
rmtpp_params

In [ ]:
model = rmtpp(rmtpp_params)

In [ ]:
model.train(train_loader, test_loader)

#Mimic2 Dataset

In [ ]:
from module.datasets import thpDataloader
from module.config import thp_data_params

In [ ]:
thp_data_params

In [ ]:
mimic = thpDataloader(thp_data_params)

In [ ]:
trainloader, testloader, num_types = mimic.load_data()

In [ ]:
tset, vset = thpDataloader.process_rmtpp(trainloader, testloader, mn_size = 4)

In [ ]:
from module.models import rmtpp
from module.config import rmtpp_params

In [ ]:
rmtpp_params['event_class'] = num_types+1
rmtpp_params['epochs'] = 2
rmtpp_params

In [ ]:
model = rmtpp(rmtpp_params)

In [ ]:
model.train(tset, vset)

#PilotElevator Dataset

In [ ]:
from module.datasets import nhpDatareader, thpDataloader
from module.config import nhp_dataset_params

In [ ]:
nhp_dataset_params

{'PathData': './data/'}

In [ ]:
data = nhpDatareader(nhp_dataset_params)

In [ ]:
train_data, val_data, total_event_num = data.read_data()

In [ ]:
from module.misc import get_dataloader

In [ ]:
trainloader = get_dataloader(train_data, batch_size=4, shuffle=True)
testloader = get_dataloader(val_data, batch_size=4, shuffle=False)

In [ ]:
tset, vset = thpDataloader.process_rmtpp(trainloader, testloader, mn_size = 4)

In [ ]:
from module.models import rmtpp
from module.config import rmtpp_params

In [ ]:
rmtpp_params['event_class'] = total_event_num+1
rmtpp_params['epochs'] = 2
rmtpp_params

{'model': 'rmtpp',
 'seq_len': 10,
 'emb_dim': 10,
 'hid_dim': 32,
 'mlp_dim': 16,
 'dropout': 0.1,
 'alpha': 0.05,
 'batch_size': 1024,
 'event_class': 11,
 'verbose_step': 350,
 'importance_weight': 'store_true',
 'lr': 0.001,
 'epochs': 2}

In [ ]:
model = rmtpp(rmtpp_params)

In [ ]:
model.train(tset, vset)

100%|██████████| 25/25 [00:00<00:00, 953.81it/s]


pcnt=10, rcnt=1
epoch 0
time_error: 19.213621139526367, PRECISION: 0.1, RECALL: 0.51, F1: 0.16721311475409836


100%|██████████| 25/25 [00:00<00:00, 1243.89it/s]

pcnt=10, rcnt=1
epoch 1
time_error: 19.219345092773438, PRECISION: 0.1, RECALL: 0.51, F1: 0.16721311475409836
